In [1]:
import re
import os
import csv
from pytubefix import YouTube
from pytubefix.cli import on_progress
from pydub import AudioSegment

In [36]:
url = "https://www.youtube.com/watch?v=VJ1QLrmfQws"
yt = YouTube(url, on_progress_callback = on_progress,use_po_token=True)
video_title = yt.title
video_title = video_title.strip().replace(" ", "_").replace("/", "_")
print(video_title.strip().replace(" ", "_").replace("/", "_"))

CUDA_Memory_Model


In [6]:
ys = yt.streams.get_audio_only().download(mp3=True,output_path="/Users/srimanthdhondy/Programs/IITR/Project_3/full_length_audio_files")

In [4]:
caption = yt.captions.get_by_language_code('en')
name = "captions"+"_"+video_title+".txt"
caption_save_path = "/Users/srimanthdhondy/Programs/IITR/Project_3/captions_for_all_audio_files"
caption.save_captions(f"{caption_save_path}/{name}")


/var/folders/3y/3y2j958n5mz9_rx5c56ffm5r0000gn/T/ipykernel_1450/2520716123.py:1: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('en')


In [53]:
# Function to parse the SRT file
def parse_srt(file_path):
    with open(file_path, 'r') as f:
        srt_content = f.read()
    
    pattern = r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})'
    matches = re.findall(pattern, srt_content)
    
    timestamps = []
    for start, end in matches:
        timestamps.append({
            "start": convert_srt_timestamp_to_milliseconds(start),
            "end": convert_srt_timestamp_to_milliseconds(end)
        })
    return timestamps

# Helper function to convert SRT timestamp to milliseconds
def convert_srt_timestamp_to_milliseconds(timestamp):
    hours, minutes, seconds = timestamp.split(":")
    seconds, milliseconds = seconds.split(",")
    
    total_milliseconds = (
        int(hours) * 3600 * 1000 + 
        int(minutes) * 60 * 1000 + 
        int(seconds) * 1000 + 
        int(milliseconds)
    )
    return total_milliseconds

# Function to split audio and save to user-specified directory
def split_audio_by_timestamps(audio_file, timestamps, output_dir, starting_index):
    # Load the audio file
    audio = AudioSegment.from_file(audio_file)
    
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Split audio using parsed timestamps from the SRT file
    for i, segment in enumerate(timestamps, start=starting_index):
        start_time = segment['start']
        end_time = segment['end']
        
        # Extract the audio segment based on start and end time
        audio_clip = audio[start_time:end_time]
        
        # Save the audio clip with a unique name in the user-specified directory
        output_path = os.path.join(output_dir, f"audio_clip_{i}.wav")
        audio_clip.export(output_path, format="wav")
        print(f"Exported audio_clip_{i}.wav from {start_time}ms to {end_time}ms in {output_dir}")

# Main function to execute the process
def main():
    # User inputs for file paths and output directory
    srt_file = input("Enter the path to the SRT file(.txt): ")
    audio_file = input("Enter the path to the full length audio file (e.g., .mp3): ")
    output_dir = input("Enter the directory where the trimmed audio clips should be saved: ")
    
    # User input for the starting index value
    starting_index = int(input("Enter the starting value for naming audio clips: "))

    # Parse the SRT file for timestamps
    timestamps = parse_srt(srt_file)

    # Split and save the audio clips, starting from the given index
    split_audio_by_timestamps(audio_file, timestamps, output_dir, starting_index)

# Run the script
if __name__ == "__main__":
    main()


Exported audio_clip_5747.wav from 80ms to 4270ms in /Users/srimanthdhondy/Programs/IITR/Project_3/folders_of_audio_clips_of_all_full_length_audio/wavs_What_Are_APIs_-_Simply_Explained
Exported audio_clip_5748.wav from 4270ms to 8080ms in /Users/srimanthdhondy/Programs/IITR/Project_3/folders_of_audio_clips_of_all_full_length_audio/wavs_What_Are_APIs_-_Simply_Explained
Exported audio_clip_5749.wav from 8080ms to 11710ms in /Users/srimanthdhondy/Programs/IITR/Project_3/folders_of_audio_clips_of_all_full_length_audio/wavs_What_Are_APIs_-_Simply_Explained
Exported audio_clip_5750.wav from 11710ms to 16170ms in /Users/srimanthdhondy/Programs/IITR/Project_3/folders_of_audio_clips_of_all_full_length_audio/wavs_What_Are_APIs_-_Simply_Explained
Exported audio_clip_5751.wav from 16170ms to 20939ms in /Users/srimanthdhondy/Programs/IITR/Project_3/folders_of_audio_clips_of_all_full_length_audio/wavs_What_Are_APIs_-_Simply_Explained
Exported audio_clip_5752.wav from 20939ms to 25949ms in /Users/srim

In [54]:
# Function to parse the transcript line by line and extract dialogues
def extract_dialogue_from_transcript(transcript_file):
    dialogues = []
    current_dialogue = []

    with open(transcript_file, 'r') as file:
        for line in file:
            # Check if the line is a timestamp (format: 00:00:00,000 --> 00:00:04,080)
            if re.match(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}', line):
                # If there's an ongoing dialogue block, join it into a single string and add to the list
                if current_dialogue:
                    dialogues.append(' '.join(current_dialogue).strip())
                    current_dialogue = []  # Reset the current dialogue list for the next block
            elif line.strip().isdigit():
                # Skip the index numbers like 1, 2, 3...
                continue
            else:
                # Accumulate non-empty lines (transcriptions)
                if line.strip():
                    current_dialogue.append(line.strip())

        # Don't forget to add the last dialogue block (after the last timestamp)
        if current_dialogue:
            dialogues.append(' '.join(current_dialogue).strip())

    return dialogues

# Function to create the metadata.csv file
def create_metadata(audio_dir, transcript_file,starting_index):
    # Extract dialogues from the transcript file
    dialogues = extract_dialogue_from_transcript(transcript_file)

    # Create a list to store the metadata entries (audio_file | transcript | transcript)
    metadata = []

    # Iterate over the dialogues and pair them with corresponding audio files
    for i, dialogue in enumerate(dialogues, start=starting_index):
        audio_filename = f"audio_clip_{i}"  # Without extensions like .mp3 or .wav
        metadata.append([audio_filename, dialogue, dialogue])
    
    metadata_file_name = "metadata"+video_title+".csv"
    output_path_metadata = f"/Users/srimanthdhondy/Programs/IITR/Project_3/metadata_all_files/{metadata_file_name}"

    # Write the metadata to the CSV file
    with open(output_path_metadata, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter='|')
        writer.writerow(["path","transcription","normalized_transcription"])
        writer.writerows(metadata)

    print(f"Metadata saved to at {output_path_metadata} to {metadata_file_name}")

# Main function to execute the script
def main():
    # User inputs
    audio_dir = input("Enter the directory where the audio clips are saved: ")  # Not used directly in this script, assumed sequential naming
    transcript_file = input("Enter the path to the transcript.txt file: ")
    starting_index = int(input("Enter the starting index for naming audio clips: "))  # User-provided starting index

    # Create metadata CSV
    create_metadata(audio_dir, transcript_file, starting_index)

# Run the script
if __name__ == "__main__":
    main()


Metadata saved to at /Users/srimanthdhondy/Programs/IITR/Project_3/metadata_all_files/metadataCUDA_Memory_Model.csv to metadataCUDA_Memory_Model.csv


In [13]:
import os
import shutil
import pandas as pd

# Paths (update if needed)
base_dir = '/Users/srimanthdhondy/Programs/IITR/Project_3/'
audio_dir = os.path.join(base_dir, 'folders_of_audio_clips_of_all_full_length_audio')
metadata_dir = os.path.join(base_dir, 'metadata_all_files')
output_audio_dir = os.path.join(base_dir, 'merged_audio_clips_all')
output_metadata_file = os.path.join(base_dir, 'merged_metadata_all.csv')

# Ensure the output directory for audio files exists
if not os.path.exists(output_audio_dir):
    os.makedirs(output_audio_dir)

def merge_audio_files():
    """ Move all audio files into a single folder """
    for audio_folder in os.listdir(audio_dir):
        audio_folder_path = os.path.join(audio_dir, audio_folder)
        
        if os.path.isdir(audio_folder_path):
            print(f"Processing {audio_folder}...")
            
            # Move all audio files into the output directory
            for audio_file in os.listdir(audio_folder_path):
                shutil.move(os.path.join(audio_folder_path, audio_file), 
                            os.path.join(output_audio_dir, audio_file))

def merge_metadata_files():
    """ Merge all metadata CSV files into a single metadata file and sort by audio clip names """
    metadata_frames = []
    
    for metadata_file in os.listdir(metadata_dir):
        metadata_file_path = os.path.join(metadata_dir, metadata_file)
        
        if metadata_file.endswith('.csv'):
            print(f"Reading and merging {metadata_file}...")
            
            # Read the CSV file
            metadata_df = pd.read_csv(metadata_file_path,sep="|")
            
            # Assuming the metadata file contains a column 'audio_clip' (or similar)
            # that contains the audio clip file names. 
            # Adjust column name if needed.
            if 'path' not in metadata_df.columns:
                raise ValueError(f"'path' column not found in {metadata_file}")
            
            # Add the dataframe to the list
            metadata_frames.append(metadata_df)

    # Concatenate all metadata dataframes into one
    merged_metadata = pd.concat(metadata_frames, ignore_index=True)

    # Sort the merged metadata by the 'audio_clip' column
    merged_metadata = merged_metadata.sort_values(by='path')

    # Save the sorted and merged metadata to a single CSV file
    merged_metadata.to_csv(output_metadata_file, index=False,sep="|")
    print(f"Merged and sorted metadata saved to {output_metadata_file}")

if __name__ == "__main__":
    # Merge all audio files into one folder
    merge_audio_files()
    
    # Merge all metadata files into a single CSV and sort them by audio clip name
    merge_metadata_files()


Processing wavs_Getting_Started_With_CUDA...
Processing wavs_APIs_for_Beginners_2023_-_How_to_use_an_API_(Full_Course___Tutorial)...
Processing wavs_CUDA Developer Tools | Performance Analysis with NVIDIA Nsight Systems Timeline...
Processing wavs_What_Are_APIs?_-_Simply_Explained...
Processing wavs_CUDA_Memory_Model...
Processing wavs_What_is_a_REST_API?...
Processing wavs_CUDA_tutorials_I_installing CUDA...
Processing wavs_CUDA Developer Tools | SOL Analysis with NVIDIA Nsight Compute...
Reading and merging metadata_CUDA Developer Tools | Performance Analysis with NVIDIA Nsight Systems Timeline.csv...
Index(['path', 'transcription', 'normalized_transcription'], dtype='object')
Reading and merging metadata_CUDA Tutorials I Installing CUDA Toolkit on Windows and WSL.csv...
Index(['path', 'transcription', 'normalized_transcription'], dtype='object')
Reading and merging metadataWhat_is_a_REST_API?.csv...
Index(['path', 'transcription', 'normalized_transcription'], dtype='object')
Reading

In [55]:
import os
import pandas as pd
import re
import shutil
from pydub import AudioSegment

# Function to extract numeric value from the audio_clip name
def extract_numeric_clip_number(audio_clip_name):
    match = re.search(r'audio_clip_(\d+)', audio_clip_name)
    if match:
        return int(match.group(1))
    return -1  # In case of an invalid format

# Function to merge and sort metadata by audio clip names
def merge_and_sort_metadata(metadata_folder, output_metadata_file):
    metadata_list = []
    
    # Iterate over each metadata CSV file in the folder
    for root, dirs, files in os.walk(metadata_folder):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f"Merging metadata from {file_path}")
                
                # Read the metadata CSV file and append it to the list
                metadata_df = pd.read_csv(file_path, header=None,sep="|")
                metadata_df.columns = ['audio_clip', 'sentence_1', 'sentence_2']  # Assuming 3 columns
                metadata_list.append(metadata_df)
    
    # Concatenate all metadata DataFrames
    merged_metadata_df = pd.concat(metadata_list, ignore_index=True)
    
    # Extract numeric part from the 'audio_clip' column for sorting
    merged_metadata_df['clip_number'] = merged_metadata_df['audio_clip'].apply(extract_numeric_clip_number)
    
    # Sort the DataFrame based on the extracted numeric clip number
    sorted_metadata_df = merged_metadata_df.sort_values(by='clip_number').drop(columns=['clip_number'])
    
    # Save the sorted metadata to a new CSV file
    sorted_metadata_df.to_csv(output_metadata_file, index=False, header=False,sep="|")
    print(f"Sorted metadata saved to {output_metadata_file}")

# Function to convert and merge all audio clips into a single folder
def merge_and_convert_audio_clips(audio_folder, output_audio_folder):
    if not os.path.exists(output_audio_folder):
        os.makedirs(output_audio_folder)

    # Iterate over each audio file in the folder structure
    for root, dirs, files in os.walk(audio_folder):
        print(f"Checking folder: {root}")  # Debugging output
        for file in files:
            print(f"Found file: {file}")  # Print all found files
            if file.endswith(('.mp3', '.m4a', '.wav')):  # Check for supported audio formats
                src_file_path = os.path.join(root, file)
                audio_clip_name = os.path.splitext(file)[0]  # Get file name without extension
                dest_file_path = os.path.join(output_audio_folder, f"{audio_clip_name}.wav")  # Ensure .wav format
                
                # Convert to .wav if the file is not already in .wav format
                if not file.endswith('.wav'):
                    print(f"Converting {src_file_path} to {dest_file_path}")
                    audio = AudioSegment.from_file(src_file_path)
                    audio.export(dest_file_path, format="wav")
                else:
                    # Just copy if it's already a .wav file
                    print(f"Copying {src_file_path} to {dest_file_path}")
                    shutil.copy2(src_file_path, dest_file_path)
    
    print(f"All audio clips have been merged and converted to {output_audio_folder}")

# Main function to merge metadata and audio files
def main():
    # Paths for metadata and audio clips
    metadata_folder = '/Users/srimanthdhondy/Programs/IITR/Project_3/metadata_all_files/'  # Update with actual metadata folder path
    audio_folder = '/Users/srimanthdhondy/Programs/IITR/Project_3/folders_of_audio_clips_of_all_full_length_audio/'  # Update with actual audio folder path
    
    # Output paths
    output_metadata_file = '/Users/srimanthdhondy/Programs/IITR/Project_3/merged_metadata.csv'  # Path to save merged metadata
    output_audio_folder = '/Users/srimanthdhondy/Programs/IITR/Project_3/merged_audio_clips/'  # Folder to save merged audio clips

    # Merge and sort metadata
    merge_and_sort_metadata(metadata_folder, output_metadata_file)

    # Merge and convert audio clips
    merge_and_convert_audio_clips(audio_folder, output_audio_folder)

# Run the script
if __name__ == "__main__":
    main()


Merging metadata from /Users/srimanthdhondy/Programs/IITR/Project_3/metadata_all_files/3_metadata_CUDA Tutorials I Installing CUDA Toolkit on Windows and WSL.csv
Merging metadata from /Users/srimanthdhondy/Programs/IITR/Project_3/metadata_all_files/1_metadata_CUDA_Memory_Model.csv
Merging metadata from /Users/srimanthdhondy/Programs/IITR/Project_3/metadata_all_files/8_metadata_What Are APIs - Simply Explained.csv
Merging metadata from /Users/srimanthdhondy/Programs/IITR/Project_3/metadata_all_files/2_metadata_Getting Started With CUDA for Python Programmers.csv
Merging metadata from /Users/srimanthdhondy/Programs/IITR/Project_3/metadata_all_files/6_metadata_APIs_for_Beginners_2023_-_How_to_use_an_API_(Full_Course___Tutorial).csv
Merging metadata from /Users/srimanthdhondy/Programs/IITR/Project_3/metadata_all_files/5_metadata_CUDA Developer Tools | SOL Analysis with NVIDIA Nsight Compute.csv
Merging metadata from /Users/srimanthdhondy/Programs/IITR/Project_3/metadata_all_files/4_metadat